In [1]:
import numpy as np
import math
import csv
from statistics import mode
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [2]:
# SET50 (unrealize)
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'GLOBAL', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

# remove (low data) : TPIPP WHA SPRC SAWAD PSH MTLS IVL GPSC EA CBG BPP BEAUTY PTTGC

In [56]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[2] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = (float(row[2]) + float(row[3]))/2.0
                if temp_value != 0:
                    unrealize = (temp - temp_value) / temp_value
                else:
                    unrealize = 0
                unrealize = ["{0:.4f}".format(unrealize)]
                stock_data.append(unrealize)
                temp_value = temp
                
    return stock_data

def mean_square_error(actual, predict):
    length = len(actual)
    sums_error = 0.0
    for i, j in zip(actual, predict):
        sums_error += (i - j)**2
    
    return sums_error / length

In [57]:
def findNewXTest(x_test, predict):
    for (index, element) in enumerate(x_test[0]):
        for i in range(len(element)):
            if i == 0:
                continue
            elif i < 29:
                element[i - 1] = element[i]
            else:
                element[i - 1] = element[i]
                element[i] = predict[index]
    return x_test

In [58]:
stock_datas = [ load_stock_data(x, 2007, 2017) for x in SET50 ]

min_count = len(stock_datas[0])

for index, element in enumerate(stock_datas):
    if len(element) < min_count:
        min_count = len(element)

min_count

2043

In [67]:
model = Sequential()
    
model.add(Conv2D(64, (3, 3), input_shape=(len(stock_datas), 30, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3 ,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))   
      
model.add(Flatten())  
model.add(Dense(512))  
model.add(Activation('relu'))  
model.add(Dropout(0.5))
model.add(Dense(len(stock_datas)))
model.add(Activation('softmax'))

optimizer = SGD(lr = 0.01, decay= 1e-6, momentum = 0.9, nesterov = True)

model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [68]:
rounds = min_count - 30
train_index = int(rounds * 0.8)
validate_index = train_index + int(rounds * 0.1)

In [69]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_validate = x[train_index: validate_index].astype('float32')
x_test = x[validate_index: validate_index + 1].astype('float32')

print(x_train.shape, x_validate.shape, x_test.shape)

(1610, 37, 30, 1) (201, 37, 30, 1) (1, 37, 30, 1)


In [70]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_validate = y[train_index: validate_index].astype('float32')
y_test = y[validate_index: rounds].astype('float32')

print(y_train.shape, y_validate.shape, y_test.shape)

(1610, 37) (201, 37) (202, 37)


In [71]:
epochs = 100

model.fit(x_train, y_train, validation_data = (x_validate, y_validate), epochs = epochs, shuffle = True, batch_size = 100, verbose= 1)

Train on 1610 samples, validate on 201 samples
Epoch 1/100
1610/1610 [==============================] - 1s 805us/step - loss: 0.0016 - acc: 0.0248 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 2/100
1610/1610 [==============================] - 1s 496us/step - loss: 0.0016 - acc: 0.0298 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 3/100
1610/1610 [==============================] - 1s 494us/step - loss: 0.0016 - acc: 0.0335 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 4/100
1610/1610 [==============================] - 1s 508us/step - loss: 0.0016 - acc: 0.0248 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 5/100
1610/1610 [==============================] - 1s 495us/step - loss: 0.0016 - acc: 0.0236 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 6/100
1610/1610 [==============================] - 1s 499us/step - loss: 0.0016 - acc: 0.0292 - val_loss: 9.2413e-04 - val_acc: 0.0945
Epoch 7/100
1610/1610 [==============================] - 1s 509us/step - loss: 0.0016 - acc: 0.0366 - val_l

In [72]:
for (index, element) in enumerate(y_test):
    predict = model.predict(x_test, verbose = 0)
    error = mean_square_error(element, predict[0])
    print("rounds:",index + 1, error)
    x_test = findNewXTest(x_test, predict[0])



rounds: 1 0.000665932152525
rounds: 2 0.000818042309741
rounds: 3 0.000707070523194
rounds: 4 0.000815524890859
rounds: 5 0.000696879246718
rounds: 6 0.00094520740228
rounds: 7 0.00100218357256
rounds: 8 0.000912143427204
rounds: 9 0.00128069036648
rounds: 10 0.000917052001024
rounds: 11 0.000772102954704
rounds: 12 0.00135703828994
rounds: 13 0.00111123223562
rounds: 14 0.00131835177689
rounds: 15 0.00106144493362
rounds: 16 0.000744946740437
rounds: 17 0.00116519498126
rounds: 18 0.000566749136464
rounds: 19 0.000720133273919
rounds: 20 0.000744760466603
rounds: 21 0.000740209485657
rounds: 22 0.00102340117713
rounds: 23 0.000579099127272
rounds: 24 0.000639306789878
rounds: 25 0.000844651320683
rounds: 26 0.00069323715424
rounds: 27 0.000707821405678
rounds: 28 0.000575535803083
rounds: 29 0.000803533206787
rounds: 30 0.000783245323337
rounds: 31 0.000843495276404
rounds: 32 0.00090359900804
rounds: 33 0.000626948000165
rounds: 34 0.000783757551465
rounds: 35 0.00081514500482
rounds

In [27]:
# predict